In [1]:
from ray.rllib.algorithms import ppo, a3c, dqn, apex_dqn
from ray import tune
import numpy as np
import torch

import sys
sys.path.append("../..")
from thesis.utils.utils import get_config, setup_ray, save, load
path = "D:/Master/Masterarbeit/thesis"
setup_ray(path = path, unidirectional = False)

c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\data\spaces.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable, Mapping
2022-12-02 10:45:12,636	INFO worker.py:1528 -- Started a local Ray instance.


In [2]:
env_args = dict(
    fleetsize = 2,
    max_fleetsize = 10,    
    pseudo_routing = False,
    pseudo_dispatcher = True,
    #pseudo_dispatcher_distance = 0.3,
    routing_agent_death= True,
    death_on_target = True,
    sim_config = dict(
        dispatch = True,
        routing_ma = True,
        dispatching_ma = True,
        reward_reached_target = 10, 
        #reward_reached_target_by_time = True, 
        reward_wrong_target = -0.5,
        reward_removed_for_block = -1, 
        #reward_target_distance = -0.05,
        reward_invalid= -0.2,
        block_timeout = 20,
        reward_accepted_in_station = 1,
        reward_declined_in_station = -1,
        dispatching_interval=360,
        io_quote = 0.99  ,
        availability = 0.95,
        mttr = 5*60,
    )
)

In [3]:
agv_model = dict(
    model = dict(
        custom_model = "attn_gnn_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=16,
            with_action_mask=False,
            #with_agvs=True,
            with_stations = False,
            env_type = "minimatrix",
            n_convolutions = 2
        )
    )
)
dispatcher_model = dict(
    model = dict(
        custom_model = "lin_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=16,
            with_action_mask=False,
            with_agvs=True,
            with_stations = True,
        )
    )
)

In [4]:
config, logger_creator, checkpoint_dir = get_config(
    path = path,
    batch_size=500,
    env_args = env_args, 
    agv_model = agv_model,
    train_agv = True,
    dispatcher_model=dispatcher_model, 
    train_dispatcher=True,
    env = "minimatrix",
    run_class="comparison",
    type = "dqn"
)
#trainer =  ppo.PPO(config, logger_creator=logger_creator)
#trainer = a3c.A3CTrainer(config, logger_creator=logger_creator)
trainer = dqn.DQN(config, logger_creator=logger_creator)
##trainer = apex_dqn.ApexDQN(config, logger_creator=logger_creator)

2022-12-02 10:45:18,729	INFO simple_q.py:307 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
2022-12-02 10:45:18,733	INFO algorithm.py:457 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_44813_mh3jrh2l)
  warn(f"Unzipping to temporary directory ({tmp_dir})")
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_44835_2aeuhrug)
  warn(f"Unzipping to temporary directory ({tmp_dir})")
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma

In [5]:
#trainer.save(checkpoint_dir)

In [6]:
#trainer.restore("../../models/comparison/6_30_2022-11-14_17-44-48/checkpoint_000300/checkpoint-300")

In [7]:
for j in range(10):
    for i in range(100):
        trainer.train()    
    trainer.save(checkpoint_dir)

In [ ]:
#save(trainer, "agv", "../../models/trained")